In [14]:
import h3.api.numpy_int as h3
#from h3 import h3
import folium
#from objects import Node

class CitySim:
    '''
    This class represents the area where taxis have to be dispatched
    and the size of the hexagons.
    
    :param: geoJson - a nested list of coordinates e.g.
                    geoJson = {'type':      'Polygon',
                               'coordinates': [[[40.742239, -74.008574],
                                                [40.731461, -73.982515],
                                                [40.770477, -73.950370], 
                                                [40.782619, -73.980991]]]}
    
    :param: resolution - specifies the edge size of each hexagon. Default 
                         is 9 with edge length ~ 173 meter
    '''
    __slots__ = ['geoJson', 'resolution', 'polyline', 'hexagons', 'nodes', 'city_time']
    
    def __init__(self, geoJson, resolution=9):
        self.geoJson = geoJson
        self.resolution = resolution
        self.polyline = self.geoJson['coordinates'][0]
        self.polyline.append(self.polyline[0])
        self.hexagons = list(h3.polyfill(geoJson, resolution))
        self.nodes = [Node(node_id) for node_id in self.hexagons]
        self.city_time = 0
        #self.fleet_size = None
        #self.taxis = {}
    
    def print_nodes(self):
        pass
    
    def update_city_time(self):
        self.city_time += 1
        # set city time of drivers
        for driver_id, driver in self.drivers.iteritems():
            driver.set_city_time(self.city_time)
            
    def render_base_map(self):
        polyline = self.polyline
        polyline.append(polyline[0])
        lat = [p[0] for p in polyline]
        lng = [p[1] for p in polyline]
        m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
        my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color="green")
        m.add_child(my_PolyLine)
        
        hexagons = self.hexagons
        polylines = []
        lat = []
        lng = []
        for hex in hexagons:
            polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
            # flatten polygons into loops.
            outlines = [loop for polygon in polygons for loop in polygon]
            polyline = [outline + [outline[0]] for outline in outlines][0]
            lat.extend(map(lambda v:v[0],polyline))
            lng.extend(map(lambda v:v[1],polyline))
            polylines.append(polyline)
        for polyline in polylines:
            my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color='red')
            m.add_child(my_PolyLine)
        display(m)
        
    
        
geoJson = {'type': 'Polygon',
 'coordinates': [[[40.742239, -74.008574],
                  [40.731461, -73.982515],
                  [40.770477, -73.950370], 
                  [40.782619, -73.980991]
                 ]]}
city = CitySim(geoJson)
city.hexagons[:3]

[617733123873177599, 617733123870294015, 617733123873701887]

In [6]:
class Node(object):
    __slots__ = ('node_id', 'valid_nodes', 'neighbors', 'orders', 'taxis')
    
    def __init__(self, node_id):
        self.node_id = node_id
        #self.valid_nodes = valid_nodes
        self.neighbors = list(h3.hex_ring(self.node_id))
        self.orders = []
        self.taxis = {}
        #self.idle_drivers = 0
        
    def print_node_id(self):
        print(self.node_id)
        
    def clean(self):
        self.orders = []
        self.taxis = {}
        self.idle_drivers = 0
    
    def set_neighbors(self, valid_nodes):
        '''
        Should be in the city class to avoid copying the list of valid nodes? `\~°.°~/`
        '''
        nbs = [-100 if nb not in hexagons else nb for nb in nbs]
        neighborhoods.append(nbs)
        
    def set_orders(self):
        orders = Orders()
        for order in orders:
            self.order.append(order)
    
    def status_control(self):
        pass

In [3]:
class Taxi(object):
    __slots__ = ("online", "onservice", 'order', 'node', 'city_time', '_driver_id')
    
    def __init__(self, driver_id):
        self.online = True
        self.onservice = False
        self.order = None
        self.node = None
        self.city_time = 0
        
        self._driver_id = driver_id
        
    def set_position(self, node):
        self.node = node
    
    def set_order_start(self, order):
        self.order = order
    
    def set_order_finish(self):
        self.order = None
        self.onservice = False
        
    def update_city_time(self):
        self.city_time += 1

    def set_city_time(self, city_time):
        self.city_time = city_time
    
    def set_offline(self):
        assert self.onservice is False and self.online is True
        self.online = False
        self.node.idle_driver_num -= 1
#        self.node.offline_driver_num += 1

    def set_offline_for_start_dispatch(self):
        assert self.onservice is False
        self.online = False
        
    def set_online(self):
        assert self.onservice is False
        self.online = True
        self.node.idle_driver_num += 1
#        self.node.offline_driver_num -= 1
        
    def set_online_for_finish_dispatch(self):
        self.online = True
        assert self.onservice is False
        
    def take_order(self, order):
        assert self.online == True
        self.set_order_start(order)
        self.onservice = True
        self.node.idle_driver_num -= 1
        
    def status_update(self, city):
        assert self.city_time == city.city_time
        if self.onservice is True:
            assert self.online is True
            if self.order.end_time == city.city_time:
                self.set_position(self.order.get_end_position())
                self.set_order_finish()
                self.node.add_driver(self._driver_id, self)
#                city.n_drivers += 1v
            elif self.city_time < self.order.end_time:
                pass
            else:
                raise ValueError('Driver: status_control_eachtime(): entities are in different timezones')
        

In [ ]:
import pandas as pd
#from utils import DataLoader

class Orders(object):
    __slots__ = ('start_node', 'end_node', 'start_time', 'end_time', 'duration', 'price')
    
    def __init__(self, start_node, start_time):
        self.start_node = start_node
        self.start_time = start_time
        self.orders = []
        #self.end_node = None
        #self.end_time = 0
        #self.price = 0
    
    def get_orders(self):
        orders = DataLoader(self.start_node, self.start_time)
        return orders
    
    def 
    
        
        

In [ ]:
# utils.py
class DataLoader:
    def __init__(self, )

In [6]:
geoJson = {'type': 'Polygon',
 'coordinates': [[[40.742239, -74.008574],
                  [40.731461, -73.982515],
                  [40.770477, -73.950370], 
                  [40.782619, -73.980991]]]}

polyline = geoJson['coordinates'][0]
polyline.append(polyline[0])
lat = [p[0] for p in polyline]
lng = [p[1] for p in polyline]
m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color="green")
m.add_child(my_PolyLine)

hexagons = list(h3.polyfill(geoJson, 9))
polylines = []
lat = []
lng = []
for hex in hexagons:
    polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
    # flatten polygons into loops.
    outlines = [loop for polygon in polygons for loop in polygon]
    polyline = [outline + [outline[0]] for outline in outlines][0]
    lat.extend(map(lambda v:v[0],polyline))
    lng.extend(map(lambda v:v[1],polyline))
    polylines.append(polyline)
for polyline in polylines:
    my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color='red')
    m.add_child(my_PolyLine)
display(m)

In [14]:
print("the subzone was filled with ", len(hexagons), "hexagons at resolution 9")

the subzone was filled with  130 hexagons at resolution 9


In [10]:
def visualize_hexagons(hexagons, color="red", folium_map=None):
    """
    hexagons is a list of hexcluster. Each hexcluster is a list of hexagons. 
    eg. [[hex1, hex2], [hex3, hex4]]
    """
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    
    if folium_map is None:
        m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    else:
        m = folium_map
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
        m.add_child(my_PolyLine)
    return m

In [42]:
h3_address = city.hexagons[0]
#m = visualize_hexagons(list(h3.k_ring_distances(h3_address, 4)[3]), color="purple")
#m = visualize_hexagons(list(h3.k_ring_distances(h3_address, 4)[2]), color="blue", folium_map=m)
#m = visualize_hexagons(list(h3.k_ring_distances(h3_address, 4)[1]), color="green", folium_map=m)
#m = visualize_hexagons(list(h3.k_ring_distances(h3_address, 4)), color = "red", folium_map=m)
print(h3.k_ring_distances(h3_address, 3))
#display(m)

[array([617733123873177599], dtype=uint64), array([617733123866361855, 617733123870294015, 617733123873701887,
       617733123872129023, 617733123873439743, 617733123866623999],
      dtype=uint64), array([617733123867410431, 617733123871342591, 617733123870031871,
       617733123870818303, 617733150972837887, 617733123872653311,
       617733123872915455, 617733123872391167, 617733123868459007,
       617733123869507583, 617733123866099711, 617733123865837567],
      dtype=uint64), array([617733123878682623, 617733123879206911, 617733123871080447,
       617733123871604735, 617733123870556159, 617733150972575743,
       617733150971527167, 617733150971789311, 617733123812360191,
       617733123812622335, 617733123808690175, 617733123808165887,
       617733123868721151, 617733123867934719, 617733123868983295,
       617733123876847615, 617733123867148287, 617733123866886143],
      dtype=uint64)]


In [8]:
h3_address = 617733123873177599#neighborhoods[0][3]
m = visualize_hexagons([h3_address])
display(m)

In [73]:
hexagons[:6]

['892a1008923ffff',
 '892a100d287ffff',
 '892a100d2d7ffff',
 '892a1008927ffff',
 '892a107252fffff',
 '892a10725b7ffff']

In [74]:
#redlist = ['892a1008837ffff', '892a10089cfffff', '892a1008827ffff', '892a100d243ffff']
neighborhoods = []
for hexagon in hexagons:
    nbs = list(h3.hex_ring(hexagon))
    #nbs = [-100 if nb not in hexagons else nb for nb in nbs]
    neighborhoods.append(nbs)

neighborhoods[:6]

[['892a100892fffff',
  '892a1008927ffff',
  '892a1008937ffff',
  '892a1008933ffff',
  '892a100892bffff',
  '892a100893bffff'],
 ['892a100d2bbffff',
  '892a100d28fffff',
  '892a100d04bffff',
  '892a100d297ffff',
  '892a100d2b3ffff',
  '892a100d283ffff'],
 ['892a100d2c7ffff',
  '892a100d28bffff',
  '892a100d2c3ffff',
  '892a100d2d3ffff',
  '892a100d66fffff',
  '892a100d29bffff'],
 ['892a100d4cbffff',
  '892a1008923ffff',
  '892a100d453ffff',
  '892a100d45bffff',
  '892a100892fffff',
  '892a1008937ffff'],
 ['892a1072527ffff',
  '892a1072193ffff',
  '892a1072523ffff',
  '892a100d25bffff',
  '892a1072197ffff',
  '892a107252bffff'],
 ['892a100d67bffff',
  '892a10725a3ffff',
  '892a10725b3ffff',
  '892a100d66bffff',
  '892a100d64fffff',
  '892a10725a7ffff']]

In [75]:
class Util:
    @classmethod
    def next_hex(current_hex, target_hex) -> tuple:
        '''
        Returns a tuple, containing the next hexagon to choose
        depending on the current and target location.
        '''
        neighbors = list(h3.hex_ring(current_hex))
        distances = [(nb, h3.h3_distance(nb, target_hex)) for nb in neighbors]
        distances.sort(key=lambda x: x[1])
        return distances[0]

print(next_hex('892a100893bffff', '892a100d6a3ffff'))

('892a1008923ffff', 5)


In [3]:
class Distribution():
    ''' Define the distribution from which sample the orders'''
    #__metaclass__ = ABCMeta  # python 2.7
    @abstractmethod
    def sample(self):
        pass

NameError: name 'abstractmethod' is not defined

In [4]:
class PoissonDistribution():

    def __init__(self, lam):
        self._lambda = lam

    def sample(self, seed=0):
        np.random.seed(seed)
        return np.random.poisson(self._lambda, 1)[0]

In [84]:
import numpy as np
np.random.normal(10, 2, 1)[0]

8.873823132689154

In [85]:
price = 0
price_mean = 6
price if price > 0 else price_mean

6

In [87]:
l = ['a', 'b']
l['a']

TypeError: list indices must be integers or slices, not str

In [88]:
neighbors_layers = h3.k_ring_distances(h3_address, 3)
h3.k_ring_distances(h3_address, 3)[1]

array([617733123866361855, 617733123870294015, 617733123873701887,
       617733123872129023, 617733123873439743, 617733123866623999],
      dtype=uint64)